In [68]:
import numpy as np
import pandas as pd
import impyute as imp
from scipy.stats import skewnorm

In [141]:
n = 5

arr = np.random.uniform(high=6, size=(n, n))

In [142]:
for _ in range(3):
    arr[np.random.randint(n), np.random.randint(n)] = np.nan


In [143]:
arr.shape

(5, 5)

In [144]:
cols = ['col1', 'col2', 'col3', 'col4', 'col5']

In [145]:
df = pd.DataFrame(arr, columns=cols)

In [146]:
df

,col1,col2,col3,col4,col5
0,4.403951,5.533456,1.397873,1.809761,1.574816
1,3.813936,3.170346,3.884659,1.992007,NaN
2,NaN,3.543530,5.668018,4.877539,4.799607
3,4.806383,NaN,2.722659,1.811484,1.547786
4,3.959935,4.855701,3.273549,5.075153,4.139866


In [147]:
df_data = imp.mice(df)

/home/igor/.local/share/virtualenvs/scripts-I1e1_A0A/lib/python3.6/site-packages/impyute/util/preprocess.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  args[0] = args[0].as_matrix()


In [148]:
df_data = pd.DataFrame(df_data.values, columns=list(df))

In [149]:
df_data

,col1,col2,col3,col4,col5
0,4.403951,5.533456,1.397873,1.809761,1.574816
1,3.813936,3.170346,3.884659,1.992007,4.357931
2,4.337188,3.543530,5.668018,4.877539,4.799607
3,4.806383,5.077487,2.722659,1.811484,1.547786
4,3.959935,4.855701,3.273549,5.075153,4.139866


In [150]:
def skewy(val, maxi, mini):
    return (val - mini)/(maxi - mini)

In [151]:
def geek(val, mean, std):
    normed = (val - mean)/std
    return skewnorm.pdf(normed, a=-5, loc=mean, scale=5 * std)

In [152]:
def threshold(val, mean, std):
    if val < (mean - 2 * std):
        return mean - 2 * std
    elif val > (mean + 2 * std):
        return mean + 2 * std
    else:
        return val

In [153]:
df_data['col1'] = df_data['col1'].apply(threshold, mean=df_data['col1'].mean(), std=df_data['col1'].std())
df_data['col2'] = df_data['col2'].apply(threshold, mean=df_data['col2'].mean(), std=df_data['col2'].std())
df_data['col3'] = df_data['col3'].apply(threshold, mean=df_data['col3'].mean(), std=df_data['col3'].std())
df_data['col4'] = df_data['col4'].apply(threshold, mean=df_data['col4'].mean(), std=df_data['col4'].std())
df_data['col5'] = df_data['col5'].apply(threshold, mean=df_data['col5'].mean(), std=df_data['col5'].std())

In [154]:
df_data['col1'] = df_data['col1'].apply(skewy, maxi=df_data['col1'].max(), mini=df_data['col1'].min())
df_data['col2'] = df_data['col2'].apply(skewy, maxi=df_data['col2'].max(), mini=df_data['col2'].min())
df_data['col3'] = df_data['col3'].apply(skewy, maxi=df_data['col3'].max(), mini=df_data['col3'].min())
df_data['col4'] = df_data['col4'].apply(skewy, maxi=df_data['col4'].max(), mini=df_data['col4'].min())
df_data['col5'] = df_data['col5'].apply(skewy, maxi=df_data['col5'].max(), mini=df_data['col5'].min())

In [155]:
df_data

,col1,col2,col3,col4,col5
0,0.594505,1.000000,0.000000,0.000000,0.008312
1,0.000000,0.000000,0.582366,0.055811,0.864176
2,0.527234,0.157921,1.000000,0.939482,1.000000
3,1.000000,0.807047,0.310244,0.000528,0.000000
4,0.147111,0.713194,0.439253,1.000000,0.797116


In [156]:
df_data

,col1,col2,col3,col4,col5
0,0.594505,1.000000,0.000000,0.000000,0.008312
1,0.000000,0.000000,0.582366,0.055811,0.864176
2,0.527234,0.157921,1.000000,0.939482,1.000000
3,1.000000,0.807047,0.310244,0.000528,0.000000
4,0.147111,0.713194,0.439253,1.000000,0.797116


In [157]:
values = [0.3, 0.2, 0.1, 0.2, 0.2]

In [158]:
relevancia = (df_data * values).sum(axis=1).to_frame('relevancia')

In [159]:
df_data = df_data.join(relevancia, how='left')

In [160]:
df_data.sort_values(by=['relevancia', 'col1'], ascending=False)

,col1,col2,col3,col4,col5,relevancia
2,0.527234,0.157921,1.000000,0.939482,1.000000,0.677651
4,0.147111,0.713194,0.439253,1.000000,0.797116,0.590121
3,1.000000,0.807047,0.310244,0.000528,0.000000,0.492539
0,0.594505,1.000000,0.000000,0.000000,0.008312,0.380014
1,0.000000,0.000000,0.582366,0.055811,0.864176,0.242234


In [69]:
skewnorm.pdf

<bound method rv_continuous.pdf of <scipy.stats._continuous_distns.skew_norm_gen object at 0x7f43980e8da0>>